# Amadeus Data Science Challenge

## Exercise 4. Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

- Llegados a este ejercicio se va a descubrir que todo el dataset tenía duplicados y debería haberlos eliminado
- ¿Cómo voy a descubrir esto? Pues porque el ejercicio me pide que haga un `join` para cruzar ambas tablas y al jacer un left join me va a dar muchos resultados. En este sentido, no se puede hacer un `left join` cuando la tabla de la derecha tiene duplicados, porque si los tiene mi tabla de la izquierda va a tener más resultados después del left join que antes, cuando debería tener los mismos.
    - En ese sentido, siempre después de realizar un `join` se debe hacer un **sanity check** porque un `left join` o un `right join` devuelven más resultados que los que tiene la tabla inicial es que algo se ha hecho mal

### CONCLUSIÓN: *Debería haber realizado un drop_duplicates anteriormente para ver que no tenía duplicados*

In [2]:
cd /home/dsc/Data/challenge

/home/dsc/Data/challenge


### ¿Cómo eliminar los duplicados?
- Con el código de abajo hago que todas las columnas sean de tipo string (dtype=str) y que no intente averiguar su tipo (low_memory=True)
- Voy a hacer una iteración con chunks, de tal manera que me elimine los duplicados. El funcionamiento es el siguiente:
    - Va a coger de 1.000.000 en un millón las líneas por archivo y las va a meter en un pd.DataFrame
    - Sobre ese pd.DataFrame ('all_chunks') va a hacer un `drop_duplicates` para eliminar duplicados
    - Para cada iteración me va a imprimir cuántos resultados únicos hay. Como se puede ver, la mayoría son duplicados.
    - Finalmente, se va a exportar a un csv
    - Esto se hará para ambos documentos: 'searches.csv' y 'bookings'

In [7]:
#En el siguiente código 'low_memory=False' indica que no intente predecir de qué tipo es cada columna.
#Con dtype=string indico que todas las columnas son string
import pandas as pd
fi = pd.read_csv("./searches.csv.bz2", sep="^", 
                 low_memory=False, dtype=str, chunksize=1000000)

all_chunks = pd.DataFrame()
for i, chunk in enumerate(fi):
    all_chunks = all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print(i, len(all_chunks))
    
all_chunks.to_csv("searches.no_dup.csv", sep="^", index=False)

0 358999
1 359003
2 359003
3 359003
4 359003
5 359003
6 359003
7 359003
8 359003
9 359003
10 359003
11 359003
12 359003
13 359003
14 359003
15 359003
16 359003
17 359003
18 359003
19 359003
20 359004


In [8]:
import pandas as pd
fi = pd.read_csv("./bookings.csv.bz2", sep="^", 
                 low_memory=False, dtype=str, chunksize=1000000)

all_chunks = pd.DataFrame()
for i, chunk in enumerate(fi):
    all_chunks = all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print(i, len(all_chunks))
    
all_chunks.to_csv("bookings.no_dup.csv", sep="^", index=False)

0 1000000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000003
6 1000003
7 1000003
8 1000003
9 1000003
10 1000003


### Action plan

1) seleccionar las columnasde 'bookings' 
    (dep_port, arr_port, 'cre_date           ')

2) arreglar los espacios en el código de los aeropuertos en la tabla 'bookings'

3) drop duplicates de 'bookings'

4) poner fecha en mismo formato (str)

4) crear tabla con todo 1

5) search left join con booking

6) fillna(0)

7) save file


In [39]:
b=pd.read_csv('bookings.no_dup.csv', sep="^", 
              usecols=['dep_port', 'arr_port', 'cre_date           '])
s=pd.read_csv('searches.no_dup.csv', sep="^", low_memory=False)

In [40]:
b.columns=b.columns.str.strip()
b.dep_port=b.dep_port.str.strip()
b.arr_port=b.arr_port.str.strip()

In [41]:
b.columns

Index(['cre_date', 'dep_port', 'arr_port'], dtype='object')

- Realizo otro `drop_duplicates`. ¿Por qué lo realizo si teóricamente está limpio con el bucle que hice?
    - Lo realizo porque el dataset importado tenía muchas columnas y ahora he seleccionado solo 3, por lo que es de esperar que haya duplicados en esas tres columnas al dejar información entre medias.

In [42]:
b.drop_duplicates(inplace=True)

- Añado la columna 'booked' indicando las reservas, con un valor de 1.

In [43]:
b["booked"]=1

- Para que funcione el `left join` el formato de las fechas debe coincidir también, así que las debo unificar

In [44]:
b.cre_date[0]

'2013-02-22 00:00:00'

In [45]:
s.Date[0]

'2013-01-01'

In [46]:
b.cre_date=b.cre_date.str[:10]
b.cre_date[0]

'2013-02-22'

In [47]:
b.head(5)

,cre_date,dep_port,arr_port,booked
0,2013-02-22,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
5,2013-03-20,DEN,LGA,1
7,2013-03-25,NRT,SIN,1


- Se realiza el `left join`

In [48]:
s_b=s.merge(b,
           how='left',
           left_on=['Date', 'Origin', 'Destination'],
           right_on=['cre_date', 'dep_port', 'arr_port'])

- Se realiza un sanity check para comprobar que sigue teniendo las mismas filas antes y después del `left join`

In [49]:
len(s), len(s_b)

(359004, 359004)

- Sin embargo, tiene muchas columnas...

In [50]:
s_b.shape

(359004, 49)

In [51]:
s_b.sample(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port,booked
162873,2013-06-12,20:13:26,MPT,04aafe13795233a4a8c55ed644011e6c,IT,IST,TLV,0.0,1.0,IST,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,MIL,NaN,NaN,NaN,NaN
12677,2013-01-13,21:59:45,MPT,63d35a15724f95939e6d0dd0ca39be16,DE,FRA,SPU,1.0,2.0,FRA,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,BRE,NaN,NaN,NaN,NaN
177116,2013-06-27,16:38:08,CAL,99c89d646d27ca69967c3859e0bce237,NG,LOS,LON,1.0,2.0,LOS,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LOS,NaN,NaN,NaN,NaN
18698,2013-01-19,09:07:47,MPT,d03a30f66f773a65aad66e7bafd1110f,GR,IEV,DEL,0.0,1.0,IEV,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,ATH,NaN,NaN,NaN,NaN
287146,2013-10-15,14:50:14,MPT,3561a60621de06ab1badc8ca55699ef3,US,FOR,VIX,0.0,1.0,FOR,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,0,NaN,NaN,NaN,NaN


In [52]:
s_b[ s_b['booked']==1].head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port,booked
27,2013-01-01,18:33:28,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-01,RUH,JED,1.0
40,2013-01-01,06:36:57,FFP,86f167b84e77346849f9439ae87c02a6,SA,DMM,MNL,1.0,2.0,DMM,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-01,DMM,MNL,1.0
59,2013-01-01,07:00:38,FQD,e8741eaf2fa2f71f931475d18fa72096,US,ATL,MIA,0.0,1.0,ATL,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SEA,2013-01-01,ATL,MIA,1.0
134,2013-01-01,23:30:44,MPT,5215502d9524c3183f3839b0d9a5e4f9,AU,MEL,SYD,0.0,1.0,MEL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SYD,2013-01-01,MEL,SYD,1.0
172,2013-01-01,18:34:27,MPT,fd4afff0035bec8f8e305d38804c33f6,IN,BOM,JED,1.0,2.0,BOM,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DEL,2013-01-01,BOM,JED,1.0


- Relleno los NaN como ceros

In [53]:
s_b['booked']=s_b['booked'].fillna(0)

In [54]:
s_b.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port,booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN,NaN,NaN,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,NaN,NaN,NaN,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,NaN,NaN,NaN,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,NaN,NaN,NaN,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,NaN,NaN,NaN,0.0


In [55]:
s_b=s_b.astype({'booked':int})

- Acabaron en reserva 12.778

In [60]:
s_b['booked'].sum()

12778

In [61]:
s_b.to_csv('searches_with_bookings.csv', sep="^", index=False)

In [62]:
!head ./searches_with_bookings.csv

Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice^cre_date^dep_port^arr_port^booked
2013-01-01^20:25:57^MPT^624d8c3ac0b3a7ca03e3c167e0f48327^DE^TXL^AUH^1.0^2.0^TXL^AUH^2013-01-26^D2^^AUH^TXL^2013-02-02^D2^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^FRA^^^^0
2013-01-01^10:15:33^MPT^b0af35b31588dc4ab06d5cf2986e8e02^MD^ATH^MIL^0.0^1.0^ATH^MIL^2013-01-04^^^^^^^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^KIV^^^^0
2013-01-01^18:04:49^MPT^3561a60621de06ab1badc8ca55699ef3^US^ICT